<a href="https://colab.research.google.com/github/Zentalion/Concurso-Alura/blob/main/CozinhAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
%pip install -q google-adk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
##########################################
# --- Agente 1: Buscador de receitas --- #
##########################################
def agente_buscador(receita,data_de_hoje):
  buscador=Agent(
      name='agente_buscador', model='gemini-2.0-flash', description='Agente de buscar noticias no google',
      tools=[google_search],
      instruction='''Você é um assistente pessoal de pesquisa culinária. Sua tarefa é usar a ferramenta de busca do google (google_search)
      para recuperar as receitas mais bem avaliadas de acordo com o que o usuario pedir e colocalas na forma de receita com modo de preparo. Se for um pedido direto irá retornar até 3 receitas de fontes diferentes.Se for mais genérico como por exemplo um prato de massa ou um doce, buscar 5 receitas diferentes.
      Já se for pedido uma refeição completa como por exemplo um almoço de familia ou um jantar romantico, retornar apenas um prato de entrada, um principal e uma sobremesa.
      Apenas se vier Balacubaco você irá procurar até 3 receita baseado no dia e ano da busca e aprersentar ao usuário, não perguntar as preferencias dele. Se não tiver Balacubaco não mensionar a data.
      '''
  )
  entrada_do_agente_buscador = f'Receita: {receita}\nData de hoje: {data_de_hoje}'
  lancamentos= call_agent(buscador, entrada_do_agente_buscador)
  return lancamentos

In [ ]:
################################################
# --- Agente 2: Determinar preços --- #
################################################
def agente_preco(receita, lancamentos):
    precificador = Agent(
        name="agente_preco",
        model="gemini-2.0-flash",
        description="Agente que busca os preços",
        # Inserir as instruções do Agente de Preços #################################################
        instruction="""Você é um cozinheiro proficional, especialista em pratos com bom custo beneficio. Com base na nas receitas apresentadas pelo buscador,você deve:
        usar a ferramenta de busca do google (google_search) para encontrar o preço medio de cada ingrediente, lembrando que está no Brasil. Reescrever os preços para não ficarem com a formatação da fonte. Colocar os inguedientes em lista para ter apenas um por linha""",
        tools=[google_search]
    )

    entrada_do_agente_precificador = f"receita:{receita}\nLançamentos: {lancamentos}"
    # Executa o agente
    precos = call_agent(precificador, entrada_do_agente_precificador)
    return precos

In [ ]:
######################################
# --- Agente 3: Nutrição --- #
######################################
def agente_nutricional(receita, lancamentos):
    nutricionista = Agent(
        name="agente_nutricionista",
        model="gemini-2.0-flash",
        description="Agente de verificação de valor nuticional",
        tools=[google_search],
        instruction="""
            Você é um nutricionista experiente que sabe explicar o valor nutricional de alimentos.
            Você analisa receitas e mostra o valor nutricional delas como calorias, vitaminas, minerais e sua função em uma alimentação balanceada usando o buscador do google (google_search).
            Utilize a lista de ingrtedientes das receitas fornecidas no lançamentos para determinar o valor nutricional delas. Sugira substituições saudaveis para os ingredientes sem comprometer o sabor.
            """,

    )
    entrada_do_agente_nutricional = f"Receita: {receita}\nLançamentos: {precos}"
    # Executa o agente
    nutricao = call_agent(nutricionista, entrada_do_agente_nutricional)
    return nutricao

In [ ]:
##########################################
# --- Agente 4: Escritor da receita --- #
##########################################
def agente_receita(lancamentos, precos, nutricao):
    escritor = Agent(
        name="agente_escritor",
        model="gemini-2.0-flash",
        description="Agente escritor do cardápio.",
        instruction="""
            Você é um escritor de receitas, especializado em comunicação. Você vai pegar todas as receitas encontradas pelo lancamentos e relacionar com os preços encontrados pelo precos produzindo uma receita
            que mostra os preços dos ingredientes
            Mostrará as informações que o nutricionais completas que o agente_nutricional(nutricao) obteve para cada prato de uma maneira ludica e divertida.Ao final use as informações de nutricao para propor novas receitas relacionadas.
            """,
    )
    entrada_do_agente_receita = f"Receitas: {lancamentos}\nPreço: {precos}\nNutrição:{nutricao}"
    # Executa o agente
    cardapio = call_agent(escritor, entrada_do_agente_receita)
    return cardapio

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🔪 CozinhAI, seu chef de cozinha pessoal 🥣")

# --- Obter o Receita do Usuário ---
receita = input("🔥 O que iremos preparar hoje?: ")

# Inserir lógica do sistema de agentes ###############################################

if not receita:
  print('Então vou te surpriender hoje!')
  receita = ('Balacubaco')
  lancamentos= agente_buscador(receita,data_de_hoje)
  print('Encontrada as melhores receitas!📃')
  precos = agente_preco(receita, lancamentos)
  print('Os melhores preços estão aqui!🍳')
  nutricao = agente_nutricional(receita, lancamentos)
  print('Uma refeição balenciada é muito importante!💪')
  cardapio= agente_receita(lancamentos, precos, nutricao)
  print(cardapio)
else:
  print(f'Ok! Vamos buscar algumas sugestões para os pratos.')

  lancamentos= agente_buscador(receita,data_de_hoje)
  print('Encontrada as melhores receitas!📃')
  precos = agente_preco(receita, lancamentos)
  print('Os melhores preços estão aqui!🍳')
  nutricao = agente_nutricional(receita, lancamentos)
  print('Uma refeição balenciada é muito importante!💪')
  cardapio= agente_receita(lancamentos, precos, nutricao)
  print(cardapio)

🔪 CozinhAI, seu chef de cozinha pessoal 🥣
🔥 O que iremos preparar hoje?: 
Então vou te surpriender hoje!
Encontrada as melhores receitas!📃
Os melhores preços estão aqui!🍳
Uma refeição balenciada é muito importante!💪
## Balacubaco Nutritivo: Uma Sinfonia de Sabores e Saúde!

Olá, amantes da boa mesa! Preparem seus garfos e abram seus corações (e estômagos!), pois hoje vamos embarcar em uma aventura gastronômica que vai muito além do simples ato de cozinhar. Vamos criar uma experiência, uma celebração de sabores e nutrientes que fará seus sentidos dançarem!

Esqueça o "Balacubaco" que você procurava, porque o verdadeiro balacobaco está na explosão de cores, aromas e texturas que vamos criar juntos. E o melhor de tudo? Com um preço que cabe no seu bolso!

**A Estrela da Noite: Frango Cremoso com Batatas (Um Clássico Repaginado)**

Nosso prato principal é um clássico reinventado: o Frango Cremoso com Batatas. Mas não se engane, aqui a tradição encontra a inovação, o sabor se une à saúde, e